# __WSI - ćwiczenie 1.__
### __Zagadnienie przeszukiwania i podstawowe podejścia do niego__


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [101]:
import numpy as np
from plotly import graph_objs as go
import math
import pandas as pd

RNG = np.random.default_rng()

#### __Definicje funkcji oraz ich gradientów__

In [102]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### __Algorytm najszybszego spadku__


In [103]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    """
    Implementation of gradient descent for minimalisation purposes,
    usable with n-dimensional loss functions, if a suitable gradient 
    function is provided.

    Args:
        start_point: 
        beta: 
        grad:
        stop_treshlod:
        num_iters:

    Returns:
        Returns a tuple of an array of points generated by alogrithm
        (dependiong on the gradiant function the point format may vary),
        and a boolean value which depicts whether the function has succesfully
        performed all of its iterations or reached given stop treshold. 

    Raises:
        None
    """
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        # check if algorithm reached a local minimum,
        # i.e. func has flattened out
        # Could be changed for Euclidean distance, 
        # but this should perform faster 
        if abs(theta.sum()) < stop_treshlod:
            break
        # prevent overflow errors
        if abs(theta.sum()) > 100000:
            return steps, False
        # perform algorithm step
        point = point - beta * theta
        # add point to the output array
        steps = np.append(steps, [point], 0)
    return steps, True


#### __Generowanie wykresów__


Wykres dla funkcji jednej zmiennej f(x):

In [104]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                    title_text='Gradient Descent of single variable function',
                    plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='markers', 
                            marker=dict(size=6, color=YS,               
                            colorscale='Agsunset')))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [105]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                                title_text='Gradient Descent minimalisation of double variable function')
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True))
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='markers', 
                    marker=dict(size=4, color=ZS,               
                                colorscale='Agsunset'))
    fig.show()

#### __Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku__

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [106]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for GDA steps',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                         x=list(range(len(vals))), y=vals))
fig.show()


Współczynnik odcięcia __stop_treshold__ ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Funkcja testująca współczynnik kroku __beta__ uruchamia algorytm gradientowy __num_iters__ razy dla losowych punktów w zakresie badanego obszzaru i zwraca listę punktów końcowych wywołań algorytmu.

In [115]:
def test_beta(func, func_parameter_num, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], func_parameter_num)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

Metodą inżynierską zbadałem, że optymalna wartość zmienia się w zależnosci od rozimaru powierzchni dopuszczalnej. Dlatego, oraz dlatego, że w przypadkach innych funkcji których wykresów nie znamy, korzystne będzie przebadać większy zakres współczynników __beta__. W tym celu tworzę _dataframe_ który posłuży mi do dalszej analizy. Każda kolumna owej tabeli odpowiada badanym współczynnikom __beta__, a w wierszach znajdują się informacje o wartościach zwracanych przez _GDA_ dla losowych  punktów startowych.

In [108]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(g, 2, grad_g, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [109]:
df

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
0,-26.906213,-3.403032,-41.250996,-6.467057,-6.463166,NaN,-42.334215,-42.407148,-6.856426,-6.862070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.004036,-3.168150,-41.468102,-41.757881,-42.076766,-6.617066,-6.925806,NaN,-6.843632,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-26.711133,-42.291815,-6.283520,-42.189787,-6.436577,-6.618712,NaN,-6.786627,-6.988056,-6.866979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.360791,-3.043558,-41.238146,-6.294583,-42.488429,-42.236658,NaN,NaN,-6.832393,-42.488080,...,NaN,NaN,-31.44008,NaN,NaN,NaN,NaN,NaN,NaN,-33.720899
4,7.298190,-39.186205,-5.110106,-41.743997,-1.003074,NaN,-6.729182,NaN,NaN,-6.867225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-26.190842,-40.697825,-41.100190,-41.729203,-6.434771,-42.239231,-42.608772,NaN,-42.452962,-6.872437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,10.642768,-19.969019,-42.626282,-41.765438,-6.447568,-6.567698,-42.338892,-42.433692,-42.595616,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,-26.685933,-39.944061,-41.139680,-41.745701,-6.434848,-6.667995,-42.341451,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,12.221238,-39.064738,-41.394238,-6.115677,-42.063744,-6.613456,-6.717786,-42.540630,-6.848858,-42.626837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df.describe()

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
count,100.000000,100.000000,100.000000,100.000000,100.000000,86.000000,75.000000,62.000000,62.000000,49.000000,...,2.000000,2.000000,3.000000,3.000000,1.000000,2.000000,4.000000,3.000000,5.000000,5.000000
mean,-9.449940,-20.908911,-26.137657,-23.722646,-23.598186,-22.831666,-26.603314,-25.186569,-24.677922,-27.245074,...,-19.498176,-32.644368,-16.153859,-15.711425,-41.427385,-7.005986,-24.719747,-29.528164,-12.514946,-16.838934
std,19.330932,17.611643,17.793898,17.863676,17.873907,17.843580,17.700798,17.913137,17.949687,17.813189,...,17.666620,0.000005,13.323487,13.436030,NaN,0.000048,20.454618,19.504782,7.166464,13.549045
min,-42.299510,-42.559162,-42.627669,-42.626504,-42.627676,-42.620558,-42.626795,-42.627636,-42.626696,-42.626837,...,-31.990362,-32.644371,-31.440080,-31.185649,-41.427385,-7.006020,-42.618706,-40.790166,-21.776279,-33.720899
25%,-26.802800,-39.100188,-41.229718,-41.759872,-42.071979,-42.243218,-42.350540,-42.414566,-42.463962,-42.492528,...,-25.744269,-32.644369,-20.727725,-20.064279,-41.427385,-7.006003,-42.340869,-40.789256,-18.751426,-29.457138
50%,-8.788075,-20.506947,-41.054211,-14.077101,-7.001324,-6.972093,-42.334215,-41.646099,-24.729294,-42.486415,...,-19.498176,-32.644368,-10.015370,-8.942908,-41.427385,-7.005986,-24.627162,-40.788346,-8.035098,-7.006292
75%,10.523207,-3.272052,-5.587565,-6.141152,-6.445683,-6.627066,-6.761709,-6.790872,-6.847636,-6.870421,...,-13.252082,-32.644366,-8.510748,-7.974313,-41.427385,-7.005969,-7.006040,-23.897164,-7.005962,-7.005994
max,22.261851,0.924840,1.480751,0.594292,-1.003074,-5.587165,-6.698228,-6.778373,-6.818337,-6.862070,...,-7.005989,-32.644364,-7.006127,-7.005717,-41.427385,-7.005951,-7.005957,-7.005982,-7.005962,-7.004344


Następnie rysuję wykres wartości funkcji dla punktów zwracanych przez _GDA_, dla odpowiednich wartosci współczynników __beta__. 

In [117]:
layout = go.Layout(width=700, height=500,
                title_text='Algorithm outputs in function of beta value',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()

Na tym wykresie nie widać jednak ile iteracji dla każdej wartości __beta__ nie osiągneło minimum. Wobec tego następny wykres przedstawia stosunek ilości nieudanych iteracji do wykonanych.

In [127]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()

Następnie na w danym kwantylu oraz po ustaleniu maksymalnego błędu wyznaczam najlepszy parametr __beta__.

In [129]:
quantile_value = 0.3
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.quantile(quantile_value).argmin()]
print(best_b)

0.0005


Na koniec rysuję wykres wartości osiaganych przez _GDA_ dla znalezionego parametru __beta__. 

In [130]:
layout = go.Layout(width=700, height=500,
                title_text='Best beta outputs',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()

Z powyższego wykresu można ostatecznie wnioskować, że funkcja ta ma dwa minima lokalne. Dla znalezionego parametru beta powinniśmy wobec tego otrzymać wartość minimalną w okolicy _-42_.

In [144]:
p = RNG.uniform(-3, 3, 2)
g_min = gradient_descent(p, best_b, grad_g, 0.0001, 10000)[0][-1]
print(g_min)
print(g(g_min))

[-1.41237061 -0.04993894]
-42.62761658370537


Ostatecznie, dla _10000_ iteracji algorytm znalazł minimum w punkcie